In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this


In [ ]:
from automl.meta_rl.hyperparameter_suggestion import HyperparameterSuggestion
import hp_experiment

hyperparameters_to_change : list[HyperparameterSuggestion] = hp_experiment.get_hyperparameters_to_change()

In [ ]:
configuration_dict = hp_experiment.get_configuration_dict(num_episodes=12)

In [ ]:
hp_optimization_input = hp_experiment.gen_hp_optimization_input(hyperparameters_to_change, configuration_dict)

In [ ]:
from automl.base_configurations.base_configurations import load_configuration_dict

hp_optimization_input["n_trials"] = 20
hp_optimization_input["steps"] = 2

In [ ]:
from automl.meta_rl.hp_optimization_pipeline import HyperparameterOptimizationPipeline

hp_optimization_pipeline = HyperparameterOptimizationPipeline(hp_optimization_input)

In [ ]:
hp_optimization_pipeline.run()

In [ ]:
hp_optimization_pipeline.save_configuration()

In [ ]:
experiment_path = hp_optimization_pipeline.lg.logDir

print(experiment_path)